# Ce notebook implémente l’ensemble du processus de **prétraitement** et d’**extraction de features** EEG à partir des enregistrements bruts (.set et .gdf), pour chaque patient et chaque groupe de thérapie. Il comprend :

1. **Importation et configuration**  
   - Définition des chemins de base (dossier de métadonnées, dossiers `.set` et `.gdf`).  
   - Définition des paramètres :  
     - **BANDES** spectrales (δ, θ, α, β, γ).  
     - **Électrodes standard** à extraire (17 canaux).  
     - **Paires d’asymétrie** (F3–F4, C3–C4, P7–P8).  
     - Options de **filtrage** (passe-bande 1–45 Hz) et de **rechangéchantillonnage**.

2. **Lecture et nettoyage des métadonnées**  
   - Lecture de `Tinnitus_Database.xlsx` avec en-têtes sur deux niveaux.  
   - Flatten du MultiIndex (`Session|Baseline`, etc.).  
   - Conversion en numérique des colonnes cliniques (`Age`, `HL-D`, `HL-I`, `THI`, `HAD`, …).  
   - Suppression des doublons et renommage si nécessaire.

3. **Détection des fichiers EEG**  
   - Parcours récursif des dossiers `TA_Database_set` et `TA_Database_gdf` pour lister **tous** les fichiers `.set` et `.gdf`.

4. **Fonctions d’extraction des features**  
   - **`extract_bandpower`** : calcul de la densité spectrale de puissance (PSD) via Welch, puis extraction de la **band-power** moyenne (δ→γ) pour chaque canal.  
   - **`extract_asymmetry`** : calcul de la différence (Gauche–Droite) pour chaque paire de canaux et chaque bande.  
   - **`process_file`** :  
     1. Lecture du fichier raw MNE.  
     2. Filtre passe-bande 1–45 Hz (notch 50 Hz optionnel si configuré).  
     3. Extraction simultanée des band-powers et des asymétries.  
     4. Renvoi d’un dictionnaire de features ou d’un dict vide en cas d’erreur.

5. **Boucle de traitement**  
   - Pour chaque fichier EEG :  
     - Identification du **patient**, du **groupe**, de la **session** et du **code** unique.  
     - Correspondance avec la ligne de `metadata_df` (via `patient_code`).  
     - Assemblage d’une ligne de DataFrame avec métadonnées + features EEG.

6. **Agrégation par “patient+groupe”**  
   - **GroupBy** sur `code` :  
     - Colonnes non numériques (IDs, métadonnées) conservées via `first()`.  
     - Features numériques (band-power, asymétrie) moyennées via `mean()`.  
   - Résultat : `patient_features.csv` (une ligne par patient+groupe).

7. **Sauvegarde des résultats**  
   - **`patient_features.csv`** : agrégé, prêt pour l’analyse exploratoire.  
   - **`EEG_features.npy`** : brut, non agrégé, contenant toutes les lignes et colonnes du DataFrame intermédiaire.

---

> **Note :**  
> Cette pipeline est conçue pour être modulaire et extensible :  
> - Vous pouvez ajouter une étape de **notch filter** 50 Hz en activant la variable `NOTCH = True`.  
> - Pour intégrer des **mesures de connectivité** (cohérence, PLV), ajoutez les méthodes dans `CONNECTIVITY_METRICS` puis décommentez la partie `extract_connectivity`.  
> - Pou

In [ ]:
# %% [markdown]
"""
# Notebook de Préprocessing
Ce notebook réalise :
1. Chargement et nettoyage des métadonnées.
2. Découverte des fichiers EEG (.set et .gdf).
3. Prétraitement (filtre 1–45 Hz, notch 50 Hz optionnel).
4. Extraction de features :
   - Band-power (δ, θ, α, β, γ) par canal (17 électrodes).
   - Mesures d’asymétrie (paires L/R).
   - Mesures de connectivité (cohérence, PLV) [optionnel].
5. Agrégation par patient+groupe (toutes les colonnes conservées).
6. Sauvegarde des outputs :
   - `patient_features.csv`
   - `EEG_features.npy`
"""
# %% Imports et configuration
import os
from pathlib import Path
import re

import pandas as pd
import numpy as np  
import mne
import warnings
from tqdm.notebook import tqdm



warnings.filterwarnings("ignore", category=RuntimeWarning)

mne.set_log_level('ERROR')

# %% Chemins
BASE_DIR      = Path(r"C:\Users\melon\Desktop\projet_sono\Acoustic Therapies for Tinnitus Treatment An EEG Database\Acoustic Therapies for Tinnitus Treatment An EEG Database")
GDF_DIR       = BASE_DIR / "TA_Database_gdf"
SET_DIR       = BASE_DIR / "TA_Database_set"
METADATA_PATH = BASE_DIR / "Tinnitus_Database.xlsx"

# %% Paramètres
BANDS = dict(
    delta=(1, 4), theta=(4, 8), alpha=(8, 12),
    beta=(12, 30), gamma=(30, 45),
)
STANDARD_CHANNELS = [
    "FP1","FP2","F7","F3","Fz","F4","F8",
    "T7","C3","C4","T8",
    "P7","Pz","P8",
    "O1","O2"
]
CONNECTIVITY_METRICS = []   # ex: ["coherence","plv"]
SFREQ = None                # None = conserver la sfreq d'origine
MONTAGE = None              # ex: 'standard_1020'
ASYMMETRY_PAIRS = [("F3","F4"), ("C3","C4"), ("P7","P8")]

# %% Lecture et nettoyage des métadonnées (MultiIndex + flatten)
metadata_df = pd.read_excel(
    METADATA_PATH,
    sheet_name="Database",
    header=[0, 1]
)

new_cols = []
for main, sub in metadata_df.columns:
    main = str(main).strip()
    sub  = str(sub).strip()
    if sub and not sub.lower().startswith("unnamed"):
        # on combine les deux niveaux avec un séparateur pipe
        new_cols.append(f"{main}|{sub}")
    else:
        new_cols.append(main)

metadata_df.columns = new_cols

# renommer Code → patient_code
if "Code" in metadata_df.columns:
    metadata_df = metadata_df.rename(columns={"Code": "patient_code"})

# supprimer les doublons de noms de colonnes
metadata_df = metadata_df.loc[:, ~metadata_df.columns.duplicated()]

# 2) Liste explicite des colonnes cliniques numériques
numeric_cols = [
    "Age",
    "HL-D", "HL-I",
    "THI", "THI Effect",
    "HAD-S Effect", "HAD-A Effect"
]

# 3) Conversion safe → toutes les entrées non-convertibles deviennent NaN
for c in numeric_cols:
    if c in metadata_df.columns:
        metadata_df[c] = pd.to_numeric(metadata_df[c], errors="coerce")


metadata_keys = metadata_df.columns.tolist()

# %% Fonctions utilitaires
def find_eeg_files(dirs, exts=("*.set","*.gdf")):
    files = []
    for base in dirs:
        for therapy_dir in base.iterdir():
            if not therapy_dir.is_dir(): continue
            for patient_dir in therapy_dir.iterdir():
                if not patient_dir.is_dir(): continue
                for ext in exts:
                    files.extend(patient_dir.glob(ext))
    return files

def extract_bandpower(raw, bands, channels):
    spec = raw.compute_psd(
        method='welch',
        fmin=min(b[0] for b in bands.values()),
        fmax=max(b[1] for b in bands.values()),
        n_fft=2048,
        verbose=False
    )
    psds, freqs = spec.get_data(return_freqs=True)
    bp = {}
    for name,(fmin,fmax) in bands.items():
        idx = np.where((freqs>=fmin)&(freqs<fmax))[0]
        vals = psds[:, idx].mean(axis=1)
        for ch,val in zip(raw.ch_names, vals):
            if ch in channels:
                bp[f"{name}_bp_{ch}"] = val
    return bp

def extract_asymmetry(bp, pairs):
    asym = {}
    for chL,chR in pairs:
        for band in BANDS:
            kL,kR = f"{band}_bp_{chL}", f"{band}_bp_{chR}"
            if kL in bp and kR in bp:
                asym[f"{band}_asym_{chL}_{chR}"] = bp[kL] - bp[kR]
    return asym

def process_file(fpath):
    """
    Lit un fichier .set ou .gdf et en extrait :
      - Band-power (δ,θ,α,β,γ) sur les 17 électrodes standard
      - Asymétrie G–D sur les paires définies
    Ne fait plus de découpage en epochs 2 s ni de connectivité.
    """
    try:
        # 1) Lecture
        if fpath.suffix == ".set":
            raw = mne.io.read_raw_eeglab(fpath, preload=True, verbose=False)
        else:
            raw = mne.io.read_raw_gdf(fpath, preload=True, verbose=False)

        # 2) Montage et resampling éventuel
        if MONTAGE:
            raw.set_montage(MONTAGE, on_missing="ignore")
        if SFREQ:
            raw.resample(SFREQ, npad="auto")

        # 3) Filtre passe-bande 1–45 Hz
        raw.filter(1.0, 45.0, fir_design="firwin", verbose=False)

        # 4) Extraction de la band-power directement sur tout le signal
        bp   = extract_bandpower(raw, BANDS, STANDARD_CHANNELS)

        # 5) Extraction de l’asymétrie G–D
        asym = extract_asymmetry(bp, ASYMMETRY_PAIRS)

        # 6) On ne calcule plus la connectivité (pas d'epochs)
        return {**bp, **asym}

    except Exception:
        # En cas de problème sur le fichier, on renvoie un dict vide → NaN plus tard
        return {}


# %% Extraction des features
files = find_eeg_files([SET_DIR, GDF_DIR])
id_keys   = ["patient","group","code","therapy","session"]
base_keys = id_keys + metadata_keys

rows = []
for f in tqdm(files, desc="Extraction des features", unit="fichier"):
    feats = process_file(f)
    therapy = f.parent.parent.name
    patient = f.parent.name
    session = f.stem.split("_")[2] if "_" in f.stem else None
    group   = therapy.split("-")[0]
    code    = f"{patient}{group}"
    meta    = metadata_df[metadata_df["patient_code"] == code]
    meta_d  = meta.iloc[0].to_dict() if not meta.empty else {}

    row = {k: np.nan for k in id_keys + metadata_keys}
    row.update(dict(patient=patient, group=group,
                    code=code, therapy=therapy, session=session))
    row.update(meta_d)
    row.update(feats)
    rows.append(row)

df_feats = pd.DataFrame(rows)

# %% Agrégation par patient+groupe (toutes colonnes conservées)
# Séparer clés non numériques (on prend 'first') et numériques (on prend 'mean')
non_num = id_keys + metadata_keys
num     = [c for c in df_feats.columns if c not in non_num]
agg_dict = {c: "first" for c in non_num}
agg_dict.update({c: "mean" for c in num})

patient_feats = (
    df_feats
      .groupby("code", dropna=False)
      .agg(agg_dict)
      .reset_index(drop=True)
)

# %% Sauvegarde
OUTPUT_DIR = BASE_DIR / "processed"
OUTPUT_DIR.mkdir(exist_ok=True)

patient_feats.to_csv(OUTPUT_DIR/"patient_features.csv", index=False)
np.save(OUTPUT_DIR/"EEG_features.npy", df_feats.values)


Extraction des features:   0%|          | 0/754 [00:00<?, ?fichier/s]

## Section : Nettoyage des colonnes Band-Power EEG

Ce bloc de code réalise les opérations suivantes :

1. **Chargement du CSV agrégé**  
   Lecture du fichier `patient_features.csv` généré lors du prétraitement, contenant à la fois les métadonnées cliniques et toutes les colonnes de features EEG extraites.

2. **Définition des bandes et des électrodes standards**  
   On précise les **5 bandes spectrales** (`delta`, `theta`, `alpha`, `beta`, `gamma`) et les **17 électrodes** du montage standard 10–20.

3. **Suppression des colonnes dupliquées issues de MNE**  
   Certains fichiers `.gdf` génèrent des colonnes en double avec le suffixe `_UR-2018.01`. Ces colonnes sont identifiées et supprimées pour éviter le double comptage.

4. **Identification de toutes les colonnes de band-power**  
   On récupère toutes les colonnes dont le nom contient `"_bp_"`, correspondant aux puissances spectrales par canal et par bande.

5. **Filtrage pour ne garder que les électrodes standards**  
   - On définit une fonction `is_standard_bp(col)` qui vérifie que la colonne est strictement au format `<bande>_bp_<electrode>`  
   - Et que `<bande>` appartient à la liste des bandes, et `<electrode>` à la liste des 17 canaux standards.  
   - Seules ces colonnes sont conservées, les autres (e.g. électrodes étendues) sont supprimées.

6. **Vérification rapide**  
   Impression du nombre total de colonnes supprimées (dupliquées + non-standards) et du nombre de colonnes restantes de band-power.

7. **Sauvegarde du DataFrame nettoyé**  
   Écriture en CSV du même fichier `patient_features.csv`, désormais épuré de toutes les colonnes non désirées, prêt pour l’analyse exploratoire.



In [5]:
import re
import pandas as pd

# 1) Charger le CSV
df = pd.read_csv(r"Acoustic Therapies for Tinnitus Treatment An EEG Database\Acoustic Therapies for Tinnitus Treatment An EEG Database\processed\patient_features.csv")

# 2) Définir les bandes et les 17 électrodes standard
bands = ["delta", "theta", "alpha", "beta", "gamma"]
channels = ["FP1","FP2","F7","F3","Fz","F4","F8",
            "T7","C3","C4","T8",
            "P7","Pz","P8",
            "O1","O2"]

# 3) Supprimer les duplicata MNE (“_UR-2018.01”)
ur_cols = [c for c in df.columns if "UR-2018.01" in c]
df = df.drop(columns=ur_cols)

# 4) Identifier toutes les colonnes de band-power
bp_cols = [c for c in df.columns if "_bp_" in c]

# 5) Filtrer celles qui _correspondent_ aux 17 électrodes
def is_standard_bp(col):
    """
    Renvoie True si col a la forme '<bande>_bp_<electrode>'
    ET que bande ∈ bands et electrode ∈ channels.
    """
    m = re.match(r"^(\w+)_bp_(\w+)$", col)
    if not m:
        return False
    band, ch = m.groups()
    return (band in bands) and (ch in channels)

keep_bp = [c for c in bp_cols if is_standard_bp(c)]

# 6) Droper les autres band-power (électrodes étendues)
drop_bp = [c for c in bp_cols if c not in keep_bp]
df = df.drop(columns=drop_bp)

# 7) (Optionnel) vérifier
print(f"Colonnes supprimées (UR et étendues) : {len(ur_cols) + len(drop_bp)}")
print("Il reste", len(keep_bp), "colonnes band-power :", keep_bp)

# 8) Sauvegarder le DataFrame nettoyé
df.to_csv(r"Acoustic Therapies for Tinnitus Treatment An EEG Database\Acoustic Therapies for Tinnitus Treatment An EEG Database\processed\patient_features.csv", index=False)


Colonnes supprimées (UR et étendues) : 0
Il reste 80 colonnes band-power : ['delta_bp_FP1', 'delta_bp_FP2', 'delta_bp_F7', 'delta_bp_F3', 'delta_bp_Fz', 'delta_bp_F4', 'delta_bp_F8', 'delta_bp_T7', 'delta_bp_C3', 'delta_bp_C4', 'delta_bp_T8', 'delta_bp_P7', 'delta_bp_Pz', 'delta_bp_P8', 'delta_bp_O1', 'delta_bp_O2', 'theta_bp_FP1', 'theta_bp_FP2', 'theta_bp_F7', 'theta_bp_F3', 'theta_bp_Fz', 'theta_bp_F4', 'theta_bp_F8', 'theta_bp_T7', 'theta_bp_C3', 'theta_bp_C4', 'theta_bp_T8', 'theta_bp_P7', 'theta_bp_Pz', 'theta_bp_P8', 'theta_bp_O1', 'theta_bp_O2', 'alpha_bp_FP1', 'alpha_bp_FP2', 'alpha_bp_F7', 'alpha_bp_F3', 'alpha_bp_Fz', 'alpha_bp_F4', 'alpha_bp_F8', 'alpha_bp_T7', 'alpha_bp_C3', 'alpha_bp_C4', 'alpha_bp_T8', 'alpha_bp_P7', 'alpha_bp_Pz', 'alpha_bp_P8', 'alpha_bp_O1', 'alpha_bp_O2', 'beta_bp_FP1', 'beta_bp_FP2', 'beta_bp_F7', 'beta_bp_F3', 'beta_bp_Fz', 'beta_bp_F4', 'beta_bp_F8', 'beta_bp_T7', 'beta_bp_C3', 'beta_bp_C4', 'beta_bp_T8', 'beta_bp_P7', 'beta_bp_Pz', 'beta_bp_P8', 